# Importing Data and Preprocessing (Same in All notebooks)

In [1]:
import numpy as np
import pandas as pd
import string

In [2]:
data = pd.read_csv('/users/rohanchitte/downloads/Dataset_lyrics.csv_lyrics.csv')

In [3]:
filtered = data[data['lyrics'].notnull()]
filtered

,index,song,year,artist,genre,lyrics
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu..."
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy,\nit's like you see..."
2,2,honesty,2009,beyonce-knowles,Pop,If you search\nFor tenderness\nIt isn't hard t...
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote..."
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."
...,...,...,...,...,...,...
362232,362232,who-am-i-drinking-tonight,2012,edens-edge,Country,"I gotta say\nBoy, after only just a couple of ..."
362233,362233,liar,2012,edens-edge,Country,I helped you find her diamond ring\nYou made m...
362234,362234,last-supper,2012,edens-edge,Country,Look at the couple in the corner booth\nLooks ...
362235,362235,christ-alone-live-in-studio,2012,edens-edge,Country,When I fly off this mortal earth\nAnd I'm meas...


In [4]:
import nltk
from nltk.corpus import stopwords

cleaned = filtered.copy()

# Remove punctuation
cleaned['lyrics'] = cleaned['lyrics'].str.replace("[-\?.,\/#!$%\^&\*;:{}=\_~()]", ' ')

# Remove song-related identifiers like [Chorus] or [Verse]
cleaned['lyrics'] = cleaned['lyrics'].str.replace("\[(.*?)\]", ' ')
cleaned['lyrics'] = cleaned['lyrics'].str.replace("' | '", ' ')
cleaned['lyrics'] = cleaned['lyrics'].str.replace('x[0-9]+', ' ')

# Remove all songs without lyrics (e.g. instrumental pieces)
cleaned = cleaned[cleaned['lyrics'].str.strip().str.lower() != 'instrumental']

# Remove any songs with corrupted/non-ASCII characters, unavailable lyrics
cleaned = cleaned[~cleaned['lyrics'].str.contains(r'[^\x00-\x7F]+')]
cleaned = cleaned[cleaned['lyrics'].str.strip() != '']
cleaned = cleaned[cleaned['genre'].str.lower() != 'not available']

#Selecting Pop, Rock, Country, Jazz
cleaned = cleaned.loc[(cleaned['genre'] == 'Pop') | 
            (cleaned['genre'] == 'Country') |
            (cleaned['genre'] == 'Rock') |
            (cleaned['genre'] == 'Hip-Hop') |
            (cleaned['genre'] == 'Jazz') ]
cleaned.reset_index(inplace = True)

cleaned
print(len(cleaned))

from nltk.corpus import stopwords
stop = stopwords.words('english')
#removing stop words from lyrics

cleaned['lyrics'] = cleaned['lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#lemmatizing lyrics
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text, flg_lemm=True):
    #Convert string to list (tokenize)
    lst_text = text.split()

    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
        
    ## back to string from list
    text = " ".join(lst_text)
    return text
    
#cleaned["lyrics"] = cleaned["lyrics"].apply(lemmatize_text)

cleaned["lyrics"]  = cleaned["lyrics"].apply(lambda x:  lemmatize_text(x))

df = cleaned.drop(labels=["level_0", "index","song","year","artist"], axis=1)

185493


# Splitting Data into training and test set

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

In [26]:
df_train.reset_index() 
df_test.reset_index() 

,index,genre,lyrics
0,35835,Jazz,I dance ask I dance ask I dance madame My hear...
1,2538,Hip-Hop,Sonic boom head dread cause he's tread Upon Fl...
2,63159,Rock,If I could turn page In time I'd rearrange Jus...
3,6483,Rock,record stop stop skipping equipped stor ear fu...
4,15496,Hip-Hop,Hey yeah ya know I like playersNo Diggity No d...
...,...,...,...
61208,10254,Hip-Hop,We're never done found place belong Don't stan...
61209,31630,Country,It's fake hoax nowhere road one go anywhere an...
61210,107267,Rock,I've spent much time throwing rock window That...
61211,67806,Rock,You're lookin fine long time I still remember ...


In [27]:
#train_test split
x_tr, y_tr = df_train['lyrics'].values, df_train['genre'].values
x_val, y_val = df_test['lyrics'].values, df_test['genre'].values

# Function to calculcate max length of the sequence in the corpus

In [28]:
def get_max_length(df_train):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to LSTM cell
    """
    max_length = 0
    for row in df_train['lyrics']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length


In [30]:
maximumlen = get_max_length(df_train)
maximumlen

3666

# One hot encoding genres

In [31]:
def genre_encode(genre):
    """
    return one hot encoding for Y value
    """
    if genre == 'Pop':
        return [1,0,0,0,0]
    elif genre == 'Rock':
        return [0,1,0,0,0]
    elif genre == 'Country':
        return [0,0,1,0,0]
    elif genre == 'Hip-Hop':
        return [0,0,0,1,0]
    else:
        return [0,0,0,0,1]

In [32]:
genres = df_train['genre'].tolist()
y_tr = [genre_encode(genre) for genre in genres]

In [33]:
genres = df_test['genre'].tolist()
y_val = [genre_encode(genre) for genre in genres]

# Tokenization and Padding of the sequences to make their length same

In [34]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#Tokenize the sentences
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(x_tr))

#converting text into integer sequences
x_tr_seq  = tokenizer.texts_to_sequences(x_tr) 
x_val_seq = tokenizer.texts_to_sequences(x_val)

#padding to prepare sequences of same length
x_tr_seq  = pad_sequences(x_tr_seq, maxlen=maximumlen)
x_val_seq = pad_sequences(x_val_seq, maxlen=maximumlen)

# Loading Pretrained Glove Word Embedding

In [35]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/users/rohanchitte/glove.6B.300d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [36]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

220815


# Creating a weight matrix for words in training docs

In [37]:

embedding_matrix = np.zeros((size_of_vocabulary, 300))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# LSTM Model 

In [38]:
#deep learning library
from keras.models import *
from keras.layers import *
from keras.callbacks import *

In [39]:
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,weights=[embedding_matrix],input_length=maximumlen,trainable=False)) 

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(5,activation='softmax')) 

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["accuracy"]) 

#Adding callbacks
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  

#mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3666, 300)         66244500  
_________________________________________________________________
lstm_1 (LSTM)                (None, 3666, 128)         219648    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 66,472,729
Trainable params: 228,229
Non-trainable params: 66,244,500
_________________________________________________________________
None


In [40]:
history = model.fit( np.array(x_tr_seq), np.array(y_tr), batch_size=128, epochs=5, validation_data=(np.array(x_val_seq),np.array(y_val)))

Epoch 1/5
971/971 [==============================] - 57327s 59s/step - loss: 0.9087 - accuracy: 0.6608 - val_loss: 0.8444 - val_accuracy: 0.6809
Epoch 2/5
971/971 [==============================] - 54529s 56s/step - loss: 0.8149 - accuracy: 0.6924 - val_loss: 0.8185 - val_accuracy: 0.6884
Epoch 3/5
971/971 [==============================] - 74745s 77s/step - loss: 0.7791 - accuracy: 0.7066 - val_loss: 0.7951 - val_accuracy: 0.7013
Epoch 4/5
971/971 [==============================] - 60208s 62s/step - loss: 0.7513 - accuracy: 0.7172 - val_loss: 0.8008 - val_accuracy: 0.7010
Epoch 5/5
971/971 [==============================] - 60168s 62s/step - loss: 0.7281 - accuracy: 0.7261 - val_loss: 0.7911 - val_accuracy: 0.7045


In [41]:
_,val_acc = model.evaluate(np.array(x_val_seq),np.array(y_val))
print(val_acc)

1913/1913 [==============================] - 11597s 6s/step - loss: 0.7911 - accuracy: 0.7045
0.7045398950576782


In [42]:
# serialize weights to HDF5
model.save_weights("lyrics-5-categories-model-glove.h5")
print("Saved model to disk")

Saved model to disk
